#Few- Shot Learning: How many labels do we need to achieve a good performance on unknown data?

### Install all required libraries

In [3]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 30.6 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow
   

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install segmentation_models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.2 MB/s eta 0:00:00


In [6]:
!pip install flammkuchen

In [7]:
! pip install albumentations

In [8]:
!pip install deepd3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 25.2 MB/s eta 0:00:00


In [9]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

# Neural network libraries
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE


from tensorflow import keras
import segmentation_models as sm

# Plotting
import matplotlib.pyplot as plt
#get_ipython().run_line_magic('matplotlib', 'inline')


Segmentation Models: using `tf.keras` framework.


### Function to generate triplets to use the Triplet Loss method

In [12]:
import numpy as np

def Generate_triplets(X_training, Y1_training, Y2_training, ntripletsperimage=10):
    """
    Generate and shuffle triplets for the training of the model.

    :param X_training: Images containing dendrites and spines
    :param Y1_training:  Labels for the dendrites
    :param Y2_training:  Labels for the spines
    :param ntripletsperimage:  Number of triplets to be done for each anchor image.

    """
    X_triplets = []
    Y1_triplets = []
    Y2_triplets = []
    for i in range(X_training.shape[0]):
        choosen=[] # Remember the negative images that have been selected for current image
        for j in range(ntripletsperimage):
            # Select a random image that should not be selected before and different from the current image
            idx = np.random.randint(0, X_training.shape[0]) # Randomly choose an index for the negative image
            while idx == i or idx in choosen:
                idx = np.random.randint(0, X_training.shape[0]) #Randomly choose an index for the negative image
            # Remember which negative image has been selected
            choosen.append(idx)
            # Add the triplet
            X_triplets.append(X_training[i])
            Y1_triplets.append(Y1_training[i])
            Y2_triplets.append(Y2_training[i])
            X_triplets.append(X_training[i]) # TO DO: Consider augmentation of positive
            Y1_triplets.append(Y1_training[i])
            Y2_triplets.append(Y2_training[i])
            X_triplets.append(X_training[idx])
            Y1_triplets.append(Y1_training[idx])
            Y2_triplets.append(Y2_training[idx])

    # Convert the lists into arrays
    X_triplets=np.array(X_triplets)
    Y1_triplets=np.array(Y1_triplets)
    Y2_triplets=np.array(Y2_triplets)

    ## Shuffle the triplets
    # Step 1: Reshape the array to group triplets
    grouped = X_triplets.reshape(-1, 3, 128, 128,1)
    # Step 2: Generate random permutation
    permutation = np.random.permutation(grouped.shape[0])
    # Step 3: Use the permutation to shuffle the grouped data
    randomized_grouped = grouped[permutation]
    # Step 4: Reshape back to original shape
    X_triplets_randomized = randomized_grouped.reshape(-1, 128, 128, 1)
    # Step 5: Use the same permutation to shuffle the labels
    #   First dendrite labels Y1 by triplets
    grouped_labels = Y1_triplets.reshape(-1, 3, 128, 128,1)
    randomized_grouped_labels = grouped_labels[permutation]
    Y1_randomized = randomized_grouped_labels.reshape(-1, 128, 128, 1)
    #   First spine labels Y2 by triplets
    grouped_labels = Y2_triplets.reshape(-1, 3, 128, 128,1)
    randomized_grouped_labels = grouped_labels[permutation]
    Y2_randomized = randomized_grouped_labels.reshape(-1, 128, 128, 1)

    return X_triplets_randomized, Y1_randomized, Y2_randomized



### Triplet losses implementation for dendrites and spines

In [13]:
import tensorflow as tf
from tensorflow.keras.losses import Loss, MSE, MeanSquaredError
import segmentation_models as sm


class Triplet_loss_function_dendrite(Loss):
    def __init__(self, l = 0, margin = 0):
        super(Triplet_loss_function_dendrite, self).__init__()
        self.l = l
        self.margin = margin

    def call(self, y_true, y_pred):
        """
        Computes the Triplet Loss for dendrites

        :param y_true: Ground truth labels for dendrites
        :param y_pred:  Predicted segmentation of dendrites

        """
        # Reshape ground truth and prediction in triplet format
        y_p_reshaped = tf.reshape(y_pred, (-1, 3, 128, 128, 1)) # batch, triplet (anchor, positive, negative), x, y, channel
        y_t_reshaped = tf.reshape(y_true, (-1, 3, 128, 128, 1))
        original_loss = sm.losses.dice_loss(y_t_reshaped[:,0,:,:,:], y_p_reshaped[:,0,:,:,:]) # Compute Dice Loss for the anchor
        mask = tf.not_equal(tf.squeeze(y_t_reshaped[:,0,:,:]),tf.squeeze(y_t_reshaped[:,2,:,:])) # Check which pixels in the labels are different in the anchor and the negative
        Aux = sm.losses.dice_loss(y_p_reshaped[:,0,:,:], y_p_reshaped[:,2,:,:]) # Calculate the Dice Loss between the prediction for anchor and negative
        Aux =tf.where(mask, Aux, tf.zeros_like(Aux)) # if pixels are different, write in that position the Dice Loss, if they are the same, write 0.
        triplet_loss=tf.math.maximum(tf.convert_to_tensor(0,dtype='float32'),
                    tf.reduce_mean(MSE(y_p_reshaped[:,0,:,:], y_p_reshaped[:,1,:,:])) - # Mean MSE between anchor and positive
                    tf.reduce_mean(Aux) # Mean of dice loss for pixels labeled different in anchor and negative
                    +self.margin)
        return original_loss+ self.l*triplet_loss


class Triplet_loss_function_spine(Loss):
    def __init__(self, l = 0, margin = 0):
        super(Triplet_loss_function_spine, self).__init__()
        self.l = l
        self.margin = margin

    def call(self, y_true, y_pred):
        """
        Computes the Triplet Loss for spines

        :param y_true: Ground truth labels for spines
        :param y_pred:  Predicted segmentation of spines

        """
        # Reshape output in triplet format for ground truth and prediction
        y_t_reshaped = tf.reshape(y_true, (-1, 3, 128, 128, 1)) #batch, triplet (anchor, positive, negative), x, y, channel
        y_p_reshaped = tf.reshape(y_pred, (-1, 3, 128, 128, 1))
        original_loss = MSE(y_t_reshaped[:,0,:,:,:], y_p_reshaped[:,0,:,:,:]) # Compute Mean Square Error for the anchor
        mask = tf.not_equal(tf.squeeze(y_t_reshaped[:,0,:,:]),tf.squeeze(y_t_reshaped[:,2,:,:])) # Check which pixels in the labels are different in the anchor and the negative
        Aux = MSE(y_p_reshaped[:,0,:,:], y_p_reshaped[:,2,:,:]) # Calculate the Mean Square Error between the prediction for anchor and negative
        Aux =tf.where(mask, Aux, tf.zeros_like(Aux)) # if pixels are different, write in that position the MSE, if they are the same, write 0.
        triplet_loss=tf.math.maximum(tf.convert_to_tensor(0,dtype='float32'),
                    tf.reduce_mean(MSE(y_p_reshaped[:,0,:,:], y_p_reshaped[:,1,:,:])) - # Mean MSE between anchor and positive
                    tf.reduce_mean(Aux) # Mean of MSE for pixels labeled different in anchor and negative
                    +self.margin)
        return tf.reduce_mean(original_loss)+self.l*triplet_loss


###Define the parameters and train the model

In [ ]:
import os
import numpy as np
os.environ["SM_FRAMEWORK"] = "tf.keras"
import argparse

import segmentation_models as sm
from deepd3.model import DeepD3_Model
from deepd3.training.stream import DataGeneratorStream
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

# Access the arguments
generator_data = 'DataGeneratorStreamSiamese'
samples_per_epoch = 5000
batchsize = 32
ntripletsperimage = 1
EPOCHS = 100
loss1 = 'TripletLoss'
loss2 =  'TripletLoss'
margin1 = 10
margin2 = 1
l1 = 0.05
l2 = 0.05
lr = 0.001
epoch_decay =10
value_decay = 0.025
number_runs = 1
data_path = "/content/drive/MyDrive/."
data_results = "/content/drive/MyDrive/."
augmentation = False
augmentation = (augmentation == "True")


###########  1. Load the training data  ###########

TRAINING_DATA_PATH = r"/content/drive/MyDrive/DeepD3_Training.d3set"

VALIDATION_DATA_PATH = r"/content/drive/MyDrive/DeepD3_Validation.d3set"

for i in range(number_runs):
    print(f"Run number: {i}")

    if generator_data == "DataGeneratorStream":
        factor = 1 # relation between batch size from generator and training
        dg_training = DataGeneratorStream(TRAINING_DATA_PATH, #TRAINING_DATA_PATH,
                                        batch_size=batchsize, # Data processed at once, depends on your GPU
                                        target_resolution=0.094, # fixed to 94 nm, can be None for mixed resolution training
                                        augment=augmentation,
                                        samples_per_epoch=samples_per_epoch, # number of samples per epoch
                                        shuffle=False,
                                        min_content=50)

        # Storing data into arrays for batch control
        X_training = []
        Y1_training = []
        Y2_training = []
        voltes = int(samples_per_epoch/batchsize) # Computes number of batches per epoch
        for j in range(voltes): # Done due to memory issues
            print(j,end=' ')
            Xbatch, (Y1batch, Y2batch) = dg_training[j]  # for each batch get all instances with labels
            X_training.extend(Xbatch) # Add all batch into the list
            Y1_training.extend(Y1batch)
            Y2_training.extend(Y2batch)

        # Convert lists into arrays
        X_training = np.array(X_training)
        Y1_training = np.array(Y1_training)
        Y2_training = np.array(Y2_training)

        dg_validation = DataGeneratorStream(VALIDATION_DATA_PATH,
                                            batch_size=batchsize,
                                            target_resolution=0.094,
                                            min_content=50,
                                            augment=False,
                                            samples_per_epoch=samples_per_epoch, # number of samples per epoch
                                            shuffle=False)

        # Storing data into arrays for batch control
        X_validation = []
        Y1_validation = []
        Y2_validation = []
        voltes = int(samples_per_epoch/batchsize)
        for j in range(voltes):
            print(j,end=' ')
            Xbatch, (Y1batch, Y2batch) = dg_validation[j]
            X_validation.extend(Xbatch)
            Y1_validation.extend(Y1batch)
            Y2_validation.extend(Y2batch)
        X_validation = np.array(X_validation)
        Y1_validation = np.array(Y1_validation)
        Y2_validation = np.array(Y2_validation)

    else:
        # Loading data for triplet loss
        factor = 3 # relation between batch size from generator and training

        dg_training = DataGeneratorStream(TRAINING_DATA_PATH, #TRAINING_DATA_PATH,
                                        batch_size=batchsize, # Data processed at once, depends on your GPU
                                        target_resolution=0.094, # fixed to 94 nm, can be None for mixed resolution training
                                        augment=augmentation,
                                        samples_per_epoch=samples_per_epoch, # number of samples per epoch
                                        shuffle=False,
                                        min_content=50)
        X_training = []
        Y1_training = []
        Y2_training = []
        voltes = int(samples_per_epoch/batchsize)
        for j in range(voltes):
            print(j,end=' ')
            Xbatch, (Y1batch, Y2batch) = dg_training[j]
            X_training.extend(Xbatch)
            Y1_training.extend(Y1batch)
            Y2_training.extend(Y2batch)
        X_training = np.array(X_training)
        Y1_training = np.array(Y1_training)
        Y2_training = np.array(Y2_training)

        # Convert data into triplets
        X_training, Y1_training, Y2_training = Generate_triplets(X_training, Y1_training, Y2_training, ntripletsperimage=ntripletsperimage)

        dg_validation = DataGeneratorStream(VALIDATION_DATA_PATH,
                                            batch_size=batchsize,
                                            target_resolution=0.094,
                                            min_content=50,
                                            augment=False,
                                            samples_per_epoch=samples_per_epoch, # number of samples per epoch
                                            shuffle=False)

        X_validation = []
        Y1_validation = []
        Y2_validation = []
        voltes = int(1000/batchsize)
        for j in range(voltes):
            print(j,end=' ')
            Xbatch, (Y1batch, Y2batch) = dg_validation[j]
            X_validation.extend(Xbatch)
            Y1_validation.extend(Y1batch)
            Y2_validation.extend(Y2batch)
        X_validation = np.array(X_validation)
        Y1_validation = np.array(Y1_validation)
        Y2_validation = np.array(Y2_validation)

        # Convert data into triplets
        X_validation, Y1_validation, Y2_validation = Generate_triplets(X_validation, Y1_validation, Y2_validation, ntripletsperimage=1)

###########  2. Create the DeepD3 model  ###########
# Create a naive DeepD3 model with a given base filter count (e.g. 32)
m = DeepD3_Model(filters=32)
# m.summary()

# Define losses
if loss1 == "DiceLoss":
    loss_1 = sm.losses.DiceLoss()
else:
    loss_1 = Triplet_loss_function_dendrite(l=l1,margin=margin1)

if loss2 == "MSE":
    loss_2 = MeanSquaredError()
else:
    loss_2 = Triplet_loss_function_spine(l=l2,margin=margin2)


# Set appropriate training settings
m.compile(Adam(learning_rate=lr), # optimizer, good default setting, can be tuned
        [loss_1, loss_2], # triplet custom losses for dendrite and spine
        loss_weights=[1, 1], # weights for the losses
        metrics=[[sm.metrics.iou_score,loss_1], [sm.metrics.iou_score,loss_2]]) # Metrics for monitoring progress

#########  3. Fitting  the model  ###########

# Loading some training callbacks, such as adjusting the learning rate across time, saving training progress and intermediate models

def schedule(epoch, lr):
    if epoch < epoch_decay:
        return lr
    else:
        return lr * np.exp(-value_decay)

# Save best model automatically during training
mc = ModelCheckpoint(data_results+"DeepD3_model "+str(i)+".h5.keras", save_best_only=True)

# Save metrics
csv = CSVLogger(data_results+"DeepD3_model "+str(i)+".csv")

# Adjust learning rate during training to allow for better convergence
lrs = LearningRateScheduler(schedule)

# Actually train the network
h = m.fit(x=X_training,y=(Y1_training, Y2_training),
        epochs=EPOCHS,
        batch_size=int(factor*batchsize),
        validation_data=(X_validation, (Y1_validation, Y2_validation)),
        callbacks=[mc, csv, lrs],
        shuffle = False
        )

# ## Save model for use in GUI or batch processing
# This is for saving the neural network manually. The best model is automatically saved during training.
m.save(data_results+"deepd3_custom_trained_model "+str(i)+".h5")

#########  4. Plotting the training progress
import pandas as pd
import matplotlib.pyplot as plt

# Load the training history
df = pd.read_csv(data_results+"DeepD3_model "+str(i)+".csv")

# Plot the "val_spines_iou_score" column
plt.figure(figsize=(10, 6))
plt.plot(df["val_spines_iou_score"])
plt.title("Validation Spines IoU Score Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("IoU Score")
plt.grid(True)
plt.savefig(data_results+"Spines "+str(i)+".png")

# Plot the "val_dendrites_iou_score" column
plt.figure(figsize=(10, 6))
plt.plot(df["val_dendrites_iou_score"])
plt.title("Validation Dendrites IoU Score Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("IoU Score")
plt.grid(True)
plt.savefig(data_results+"Dendrites "+str(i)+".png")


Run number: 0
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 Epoch 1/100
156/156 ━━━━━━━━━━━━━━━━━━━━ 191s 786ms/step - dendrites_iou_score: 0.0848 - dendrites_triplet_loss_function_dendrite: 1.3460 - loss: 1.4702 - spines_iou_score: 0.0232 - spines_triplet_loss_function_spine: 0.1242 - val_dendrites_iou_score: 0.1684 - val_dendrites_triplet_loss_function_dendrite: 1.2155 - val_loss: 1.3070 - val_spines_iou_score: 0.0151 - val_spines_triplet_loss

# New Section